Import Dependencies

In [132]:
import urllib
from bs4 import BeautifulSoup as bs
import nltk
import pandas  as pd

In [143]:
df = pd.read_csv('news_summary.csv')
i = 0

Extract data from html files

In [144]:
url = df['read_more'][i]
gold_summary = df['text'][i]
if "indiatoday" in url or "intoday" in url:
    baseurl = "i"
elif "hindustantimes" in url:
    baseurl = "h"
elif "theguardian" in url:
    baseurl = "t"
file = urllib.urlopen(url)
html = file.read()

<h2>Parse the data to extract article text</h2>

In [145]:
soup = bs(html, 'html.parser')

if baseurl == "i":
    entries = soup.find_all('div',{'class':'description'})
elif baseurl == "t":
    entries = soup.find_all('div',{'itemprop':'articleBody'})
elif baseurl == "h":
    entries = soup.find_all('div',{'itemprop':'articlebody'})

for each in entries:
        if each.figure:
            each.figure.decompose()
content = []
for e in entries:
     content.extend(e.find_all("p"))

text = ""
for each in content:
    text = text + each.get_text() +" "
    
text = text.encode('utf-8').decode("unicode_escape").encode('ascii','ignore')
text = nltk.sent_tokenize(text)    
    
for each in text:
    print each , "\n"

The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media. 

The union territorys administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace. 

It has been decided to celebrate the festival of Rakshabandhan on August 7. 

In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues, the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said. 

To ensure that no one skipped office, an attendance report was to be sent to the government the next evening. 

The circular was withdrawn through a one-line order issued late in the evening by the UTs department of personnel and administrative reform

Start extraction

In [146]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter

In [147]:
# tokenize the article
tokenizer = RegexpTokenizer(r'\w+')
word_list=[]
for t in text:
    word_list.extend(tokenizer.tokenize(t))


# remove stop words from the article
filtered_words = [word for word in word_list if word.lower() not in stopwords.words('english')]
total_words = len(filtered_words)
words = filtered_words

# find n-gram probability for filtered words
filtered_words  = []
for each in Counter(words).items():
    filtered_words.append([each[0] , float(each[1])/float(total_words)])

# identify important words
important_words = []
for item in filtered_words:
    if item[1]>0.003:
        important_words.append(item[0])

while '. ' in text:
    text = text.replace('. ','\n') 
# calculate sentence score
while "\n" in text:
    text = text.split("\n")

print text


# sentenceScore  = []
# for sentence in text:
#     score = 0
#     count = 0
#     for word in important_words:
#         if word not in stopwords.words('english'):
#             count = count +1
#         if word in sentence and word not in stopwords.words('english'):
#             score = score + 1
#             count = count + 1
#     sentenceScore.append([sentence,float(score)/float(len(sentence))**(1/2)])

# # extract top n/5 sentences (n is total number of sentences in the text)
# sen = sentenceScore
# sentenceScore.sort(key=lambda x: x[1], reverse=True)
# print len(sentenceScore)
# cutScore = sentenceScore[len(sentenceScore)/5][1]
# print cutScore
# count = 0
# machine_summary = ""

# for sentence in sen:
#     if sentence[1]>cutScore:
#         print sentence[0] + "\n"
#         machine_summary = machine_summary + sentence[0] + " "


# print machine_summary

['The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.', 'The union territorys administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.', 'It has been decided to celebrate the festival of Rakshabandhan on August 7.', 'In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues, the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.', 'To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.', 'The circular was withdrawn through a one-line order issued late in the evening by the UTs department of personnel and administrative

In [148]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [149]:
vect = TfidfVectorizer(stop_words='english')
dtm = vect.fit_transform(text)
features = vect.get_feature_names()

In [150]:
import numpy as np

review_id = np.random.randint(0, len(text))
review_text = words
review_length = len(words)

# create a dictionary of words and their TF-IDF scores
word_scores = {}
for word in words:
#     word = word.lower()
    if word in features:
#         print word
#         print dtm[review_id, features.index(word)]
         word_scores[word] = dtm[review_id, features.index(word)]       

print word_scores
# print words with the top 5 TF-IDF scores
print 'TOP SCORING WORDS:'
top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for word, score in top_scores:
    print word

{'office': 0.0, 'issued': 0.0, 'secretary': 0.0, 'sources': 0.0, 'soldiers': 0.0, 'staff': 0.0, '24': 0.0, 'late': 0.0, 'retreat': 0.0, '2014': 0.0, 'areas': 0.0, 'administrative': 0.0, 'sent': 0.0, 'evening': 0.0, 'government': 0.0, 'personnel': 0.0, 'reforms': 0.0, 'confined': 0.0, 'lady': 0.0, 'day': 0.0, 'celebrate': 0.0, 'apart': 0.0, 'identified': 0.0, 'territorys': 0.0, 'festivities': 0.0, 'rituals': 0.0, 'offices': 0.0, 'went': 0.0, 'bond': 0.0, 'ruling': 0.47010599180007057, 'national': 0.0, 'ripped': 0.0, 'collectively': 0.0, 'decided': 0.0, 'year': 0.0, 'home': 0.0, 'said': 0.0, 'parent': 0.47010599180007057, 'culture': 0.0, 'tools': 0.0, 'celebrated': 0.0, 'told': 0.0, 'notice': 0.0, 'power': 0.0, 'refused': 0.0, 'hours': 0.0, 'backlash': 0.0, 'celebration': 0.0, 'protect': 0.0, 'involved': 0.0, 'withdrew': 0.0, 'festival': 0.0, 'connection': 0.0, 'ensure': 0.0, 'workplace': 0.0, 'social': 0.0, 'suitable': 0.0, 'asked': 0.0, 'affairs': 0.0, 'family': 0.0, 'deputy': 0.0, 'pr

In [151]:
sentence_score = []
for each in text:
    score = 0.0
#     each = each.
    for word in each.split(" "):
#         print word
        word = word.lower()
        if word in word_scores:
#             print "Hello"
            score += word_scores[word]
#             print word_scores[word]
    sentence_score.append([each,score])    
        

In [153]:
machine_summary = ""
for each in sentence_score:
    if each[1] != 0.0:
        machine_summary = machine_summary + each[0]
        print each[0],each[1]/len(each[0])
        

The RSS is the ideological parent of the ruling BJP. 0.0271214995269


In [154]:
machine_summary

'The RSS is the ideological parent of the ruling BJP.'

In [155]:
print gold_summary

The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.


In [130]:
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams

n = 1

stemmer = SnowballStemmer("english")

gold_summary_slammed = ""
for word in gold_summary.split():
    gold_summary_slammed = gold_summary_slammed+stemmer.stem(word)+" "


machine_summary_slammed = ""
for word in machine_summary.split():
    machine_summary_slammed = machine_summary_slammed+stemmer.stem(word)+" "

n_gold = ngrams(gold_summary_slammed.split(" "),n)
n_machine = ngrams(machine_summary_slammed.split(" "),n)

gold_list = []

for gram in n_gold:
    gold_list.append(gram)

machine_list = []
for gram in n_machine:
    machine_list.append(gram)

print float(len(list(set(gold_list).intersection(machine_list))))/float(len(list(set(gold_list))))

0.622222222222
